In [1]:
import os
import sys
sys.path.append("../cnn")
sys.path.append("../utils")

import torch
from torch.autograd import Variable

from ipywidgets import interact, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# import seaborn as sns; sns.set()

from cnn_md import CNNEnd2EndMultidecoder
from kaldi_data import KaldiEvalDataset

In [8]:
# Set up environment variables for the model we want to examine
# NOT necessarily the current environment variables!!

feat_dim=40
left_context=5
right_context=5

optimizer="Adam"
learning_rate="0.001" # Use string instead of float to prevent Python's auto-formatting...
l2_reg="0.0" # Use string instead of float to prevent Python's auto-formatting...
epochs=100
batch_size=64

enc_channels=[32,32]
enc_kernels=[3,3]
enc_downsamples=[2,2]
enc_fc=[256]

latent_dim=256

dec_fc=[256]
dec_channels=[32,32]
dec_kernels=[3,3]
dec_upsamples=[2,2]

decoder_classes=["clean", "dirty"]

use_batch_norm=False
use_batch_norm_str = "true" if use_batch_norm else "false"
activation="ReLU"
weight_init="xavier_uniform"

enc_channels_delim="_" + "_".join(map(str, enc_channels))
if len(enc_channels) == 0:
     # need to recreate bash join behavior with empty array
    enc_channels_delim="_"
enc_kernels_delim="_" + "_".join(map(str, enc_kernels))
if len(enc_kernels) == 0:
     # need to recreate bash join behavior with empty array
    enc_kernels_delim="_"
enc_downsamples_delim="_" + "_".join(map(str, enc_downsamples))
if len(enc_downsamples) == 0:
     # need to recreate bash join behavior with empty array
    enc_downsamples_delim="_"
enc_fc_delim="_" + "_".join(map(str, enc_fc))
if len(enc_fc) == 0:
     # need to recreate bash join behavior with empty array
    enc_fc_delim="_"
    
dec_fc_delim="_" + "_".join(map(str, dec_fc))
if len(dec_fc) == 0:
     # need to recreate bash join behavior with empty array
    dec_fc_delim="_"
dec_channels_delim="_" + "_".join(map(str, dec_channels))
if len(dec_channels) == 0:
     # need to recreate bash join behavior with empty array
    dec_channels_delim="_"
dec_kernels_delim="_" + "_".join(map(str, dec_kernels))
if len(dec_kernels) == 0:
     # need to recreate bash join behavior with empty array
    dec_kernels_delim="_"
dec_upsamples_delim="_" + "_".join(map(str, dec_upsamples))
if len(dec_upsamples) == 0:
     # need to recreate bash join behavior with empty array
    dec_upsamples_delim="_"

use_reconstruction = False
use_reconstruction_str = "true" if use_reconstruction else "false"
    
use_transformation = False
use_transformation_str = "true" if use_transformation else "false"

# loss_func = "L1Loss"
loss_func = "MSELoss"

expt_name = "LOSS_%s/RECON_%s_TRANS_%s_ENC_C%s_K%s_P%s_F%s/LATENT_%d/DEC_F%s_C%s_K%s_P%s/ACT_%s_BN_%s_WEIGHT_INIT_%s/OPT_%s_LR_%s_L2_REG_%s_EPOCHS_%d_BATCH_%d" % (loss_func,
                                                                                                                                        use_reconstruction_str,
                                                                                                                                        use_transformation_str,
                                                                                                                                        enc_channels_delim,
                                                                                                                                        enc_kernels_delim, 
                                                                                                                                        enc_downsamples_delim, 
                                                                                                                                        enc_fc_delim,
                                                                                                                                        latent_dim,
                                                                                                                                        dec_fc_delim,
                                                                                                                                        dec_channels_delim,
                                                                                                                                        dec_kernels_delim,                                                                                                                   
                                                                                                                                        dec_upsamples_delim,
                                                                                                                                        activation,
                                                                                                                                        use_batch_norm_str,
                                                                                                                                        weight_init,
                                                                                                                                        optimizer,
                                                                                                                                        learning_rate,
                                                                                                                                        l2_reg,
                                                                                                                                        epochs,
                                                                                                                                        batch_size)





time_dim = (left_context + right_context + 1)
freq_dim = feat_dim

print("Using experiment %s" % expt_name)

clean_dataset = "timit_clean"
dirty_dataset = "timit_dirty_100_rir"
# dirty_dataset = "timit_dirty_single_rir"
# dirty_dataset = "timit_gaussian_5.0"

e2e_fc = [256,256]
e2e_fc_delim="_" + "_".join(map(str, e2e_fc))
if len(e2e_fc) == 0:
     # need to recreate bash join behavior with empty array
    e2e_fc_delim="_"
e2e_activation = "ReLU"
num_phones = 2020

Using experiment LOSS_MSELoss/RECON_false_TRANS_false_ENC_C_32_32_K_3_3_P_2_2_F_256/LATENT_256/DEC_F_256_C_32_32_K_3_3_P_2_2/ACT_ReLU_BN_false_WEIGHT_INIT_xavier_uniform/OPT_Adam_LR_0.001_L2_REG_0.0_EPOCHS_100_BATCH_64


In [9]:
# Set up datasets for clean, dirty baselines (test set only)
clean_feat_dir = "/data/sls/scratch/atitus5/meng/feats/%s/test" % clean_dataset
# clean_baseline = KaldiEvalDataset(os.path.join(clean_feat_dir, "feats.scp"))
clean_baseline = KaldiEvalDataset(os.path.join(clean_feat_dir, "feats-norm.scp"))

dirty_feat_dir = "/data/sls/scratch/atitus5/meng/feats/%s/test" % dirty_dataset
# dirty_baseline = KaldiEvalDataset(os.path.join(dirty_feat_dir, "feats.scp"))
dirty_baseline = KaldiEvalDataset(os.path.join(dirty_feat_dir, "feats-norm.scp"))

print("Set up baseline test datasets")

Set up baseline test datasets


## IF DATA AUGMENTATION NOT RUN YET

In [10]:
checkpoint_dir = "/data/sls/scratch/atitus5/meng/models/cnn/%s/%s" % (dirty_dataset,
                                                                      expt_name)
checkpoint_dir = "%s/END2END_FC_%s_ACT_%s" % (checkpoint_dir, e2e_fc_delim, e2e_activation)

# USE ONLY DECODER CLASS CLEAN
checkpoint_file = "%s/best_md_e2e.pth.tar" % checkpoint_dir

model = CNNEnd2EndMultidecoder(freq_dim=freq_dim,
                        splicing=[left_context, right_context], 
                        enc_channel_sizes=enc_channels,
                        enc_kernel_sizes=enc_kernels,
                        enc_downsample_sizes=enc_downsamples,
                        enc_fc_sizes=enc_fc,
                        latent_dim=latent_dim,
                        dec_fc_sizes=dec_fc,
                        dec_channel_sizes=dec_channels,
                        dec_kernel_sizes=dec_kernels,
                        dec_upsample_sizes=dec_upsamples,
                        activation=activation,
                        decoder_classes=["clean"],
                        use_batch_norm=use_batch_norm,
                        weight_init=weight_init,
                        e2e_fc_sizes=e2e_fc,
                        e2e_activation=e2e_activation,
                        num_phones=num_phones)

        
checkpoint = torch.load(checkpoint_file, map_location=lambda storage,loc: storage)
model.load_state_dict(checkpoint["state_dict"])
model.eval()

CNNEnd2EndMultidecoder(
  (encoder_conv): Sequential(
    (conv2d_0): Conv2d (1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (ReLU_0): ReLU()
    (maxpool2d_0): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), dilation=(1, 1))
    (conv2d_1): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (ReLU_1): ReLU()
    (maxpool2d_1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), dilation=(1, 1))
  )
  (encoder_fc): Sequential(
    (lin_0): Linear(in_features=5280, out_features=256)
    (ReLU_0): ReLU()
    (lin_final): Linear(in_features=256, out_features=256)
  )
  (decoder_fc_clean): Sequential(
    (ReLU_0): ReLU()
    (lin_0): Linear(in_features=256, out_features=256)
    (ReLU_final): ReLU()
    (lin_final): Linear(in_features=256, out_features=5280)
  )
  (decoder_deconv_clean): Sequential(
    (ReLU_0): ReLU()
    (maxunpool2d_0): MaxUnpool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 0))
    (conv2d_0): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 

In [11]:
def augmentFeats(model, feats):
    # Only decoder through clean decoder
    feats_numpy = feats.reshape((-1, freq_dim))
    num_frames = feats_numpy.shape[0]
    decoded_feats = np.empty((num_frames, freq_dim))
    for i in range(num_frames):
        frame_spliced = np.zeros((time_dim, freq_dim))
        frame_spliced[left_context - min(i, left_context):left_context, :] = feats_numpy[i - min(i, left_context):i, :]
        frame_spliced[left_context, :] = feats_numpy[i, :]
        frame_spliced[left_context + 1:left_context + 1 + min(num_frames - i - 1, right_context), :] = feats_numpy[i + 1:i + 1 + min(num_frames - i - 1, right_context), :]
        frame_tensor = Variable(torch.FloatTensor(frame_spliced))

        recon_frames = model.forward_decoder(frame_tensor, "clean")
        recon_frames_numpy = recon_frames.cpu().data.numpy().reshape((-1, freq_dim))
        decoded_feats[i, :] = recon_frames_numpy[left_context:left_context + 1, :]
    return decoded_feats

In [12]:
# color_map = "coolwarm"
color_map = "viridis"
def plotParallelUtts(utt_id_idx):
    fig, axarr = plt.subplots(3, 2, sharex=True)
    
    fig.set_size_inches(12, 8)
    
    # CLEAN baseline
    clean_baseline_utt_id = clean_baseline.utt_ids[utt_id_idx]
    clean_baseline_feats = clean_baseline.feats_for_uttid(clean_baseline_utt_id)
    axarr[0, 0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0, 0].imshow(np.transpose(clean_baseline_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[0, 0].set_title("CLEAN")
    
    # DIRTY baseline
    dirty_baseline_utt_id = dirty_baseline.utt_ids[utt_id_idx]
    dirty_baseline_feats = dirty_baseline.feats_for_uttid(dirty_baseline_utt_id)
    axarr[0, 1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[0, 1].imshow(np.transpose(dirty_baseline_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[0, 1].set_title("DIRTY")
    
    
    # CLEAN->CLEAN
    clean_clean_feats = augmentFeats(model, clean_baseline_feats)
    axarr[1, 0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1, 0].imshow(np.transpose(clean_clean_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[1, 0].set_title("CLEAN>CLEAN")
    
    # CLEAN recon difference
    clean_recon_diff = clean_clean_feats - clean_baseline_feats
    axarr[2, 0].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2, 0].imshow(np.transpose(clean_recon_diff), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[2, 0].set_title("CLEAN>CLEAN and CLEAN baseline difference")
    
    # DIRTY->CLEAN
    dirty_clean_feats = augmentFeats(model, dirty_baseline_feats)
    axarr[1, 1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[1, 1].imshow(np.transpose(dirty_clean_feats), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[1, 1].set_title("DIRTY>CLEAN")
    
    # DIRTY transform difference
    dirty_transform_diff = dirty_clean_feats - clean_baseline_feats
    axarr[2, 1].axis('off')    # Pretty-up the resulting output by removing gridlines
    axarr[2, 1].imshow(np.transpose(dirty_transform_diff), origin='lower', cmap=color_map, aspect='auto', interpolation='none')
    axarr[2, 1].set_title("DIRTY>CLEAN and CLEAN baseline difference")
    
    plt.tight_layout()
    
    # fig.savefig("%s_idx%d.eps" % (clean_baseline_utt_id, utt_id_idx))
    
    plt.show()

interact(plotParallelUtts, utt_id_idx=range(len(clean_baseline)))

A Jupyter Widget

<function __main__.plotParallelUtts>